# Boosting

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler


In [13]:
df=pd.read_csv("./Data/mushroom_cleaned.csv")

In [14]:
X = df.drop(columns=['class'])

In [15]:
# Chuẩn hóa
scaler = StandardScaler()
scaler.fit(X) # Tính trung bình và độ lệch chuẩn của từng đặc trưng
X = scaler.transform(X) # Trừ tb và chia cho độ lệch chuẩn

In [16]:
# Xử lý outlier

from scipy import stats

# Hàm để loại bỏ outlier bằng Z-score
def remove_outliers_zscore(df, column, threshold=2):
    z_scores = np.abs(stats.zscore(df[column]))
    return df[(z_scores < threshold)]

# Loại bỏ outlier trong các cột 'stem-width', 'stem-height'
df = remove_outliers_zscore(df, 'cap-diameter')
df = remove_outliers_zscore(df, 'stem-width')
df = remove_outliers_zscore(df, 'stem-height')

print(df.shape)  # Kích thước dữ liệu sau khi loại bỏ outlier

(46211, 9)


In [17]:
# chia tập 
X = df.drop(columns=['class'])
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0, stratify=y_train)

In [18]:
# AdaBoost với Decision Tree
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

DT = DecisionTreeClassifier(max_depth=3)
DT_AdaBoost = GridSearchCV(AdaBoostClassifier(DT, random_state=0), param_grid=param_grid)
DT_AdaBoost.fit(X_train, y_train).score(X_val, y_val)

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use t

0.983649915845155

In [19]:
y_pre = DT_AdaBoost.predict(X_test)
evaluate = classification_report(y_test, y_pre, output_dict=True)
DT_AdaBoost_df = pd.DataFrame(evaluate).transpose()
DT_AdaBoost_df

,precision,recall,f1-score,support
0,0.983302,0.984673,0.983987,2153.000000
1,0.986618,0.985419,0.986018,2469.000000
accuracy,0.985071,0.985071,0.985071,0.985071
macro avg,0.984960,0.985046,0.985003,4622.000000
weighted avg,0.985074,0.985071,0.985072,4622.000000


In [20]:
# Adaboost với KNN
KNN = KNeighborsClassifier(n_neighbors=3)
KNN_AdaBoost = GridSearchCV(AdaBoostClassifier(DT, random_state=0), param_grid=param_grid)
KNN_AdaBoost.fit(X_train, y_train).score(X_val, y_val)

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use t

0.983649915845155

In [21]:
y_pre = KNN_AdaBoost.predict(X_test)
evaluate = classification_report(y_test, y_pre, output_dict=True)
KNN_AdaBoost_df = pd.DataFrame(evaluate).transpose()
KNN_AdaBoost_df

,precision,recall,f1-score,support
0,0.983302,0.984673,0.983987,2153.000000
1,0.986618,0.985419,0.986018,2469.000000
accuracy,0.985071,0.985071,0.985071,0.985071
macro avg,0.984960,0.985046,0.985003,4622.000000
weighted avg,0.985074,0.985071,0.985072,4622.000000


In [22]:
# Gradient Boosting với mặc định sử dụng cây quyết định
GB_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [1, 3, 5, 7]
}

GB = GradientBoostingClassifier(max_depth=1,random_state=0)
# n_jobs=-1: Sử dụng tất cả các lõi CPU sẵn có để thực hiện việc tìm kiếm siêu tham số song song
GB_grid_search = GridSearchCV(GB, param_grid=GB_param_grid, n_jobs=-1)
GB_grid_search.fit(X_train, y_train).score(X_val, y_val)

0.9884587641259919

In [23]:
y_pre = GB_grid_search.predict(X_test)
evaluate = classification_report(y_test, y_pre, output_dict=True)
GB_grid_search_df = pd.DataFrame(evaluate).transpose()
GB_grid_search_df

,precision,recall,f1-score,support
0,0.989322,0.989782,0.989552,2153.000000
1,0.991086,0.990684,0.990885,2469.000000
accuracy,0.990264,0.990264,0.990264,0.990264
macro avg,0.990204,0.990233,0.990219,4622.000000
weighted avg,0.990264,0.990264,0.990264,4622.000000


In [24]:
# XGBoost 
import xgboost as xgb

# Sử dụng GPU để huấn luyện mô hình
xgb_model = xgb.XGBClassifier(use_label_encoder=False, tree_method='gpu_hist', eval_metric='logloss', random_state=0)
xgb_grid_search = GridSearchCV(xgb_model, param_grid=GB_param_grid, n_jobs=-1)
xgb_grid_search.fit(
    X_train, y_train,
    early_stopping_rounds=10, # Nếu không cải thiện sau 10 vòng lặp thì dừng
    eval_set=[(X_val, y_val)], # Đánh giá hiệu suất
    verbose=True # Hiển thị quá trình
)

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:54:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.64209
[1]	validation_0-logloss:0.60156
[2]	validation_0-logloss:0.56431
[3]	validation_0-logloss:0.53660
[4]	validation_0-logloss:0.51037
[5]	validation_0-logloss:0.48627
[6]	validation_0-logloss:0.46767
[7]	validation_0-logloss:0.44925
[8]	validation_0-logloss:0.42932
[9]	validation_0-logloss:0.40990
[10]	validation_0-logloss:0.39746
[11]	validation_0-logloss:0.38428
[12]	validation_0-logloss:0.36945
[13]	validation_0-logloss:0.35819
[14]	validation_0-logloss:0.34167
[15]	validation_0-logloss:0.33241
[16]	validation_0-logloss:0.32025
[17]	validation_0-logloss:0.31080
[18]	validation_0-logloss:0.30377
[19]	validation_0-logloss:0.29707
[20]	validation_0-logloss:0.29136
[21]	validation_0-logloss:0.28266
[22]	validation_0-logloss:0.27399
[23]	validation_0-logloss:0.26947
[24]	validation_0-logloss:0.26386
[25]	validation_0-logloss:0.25941
[26]	validation_0-logloss:0.25155
[27]	validation_0-logloss:0.24723
[28]	validation_0-logloss:0.24406
[29]	validation_0-loglos

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=0, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [1, 3, 5, 7],
                         'n_estimators': [50, 100, 200]})

In [25]:
y_pre = xgb_grid_search.predict(X_test)
evaluate = classification_report(y_test, y_pre, output_dict=True)
xgb_grid_search_df = pd.DataFrame(evaluate).transpose()
xgb_grid_search_df

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:54:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:54:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the b

,precision,recall,f1-score,support
0,0.987494,0.990246,0.988868,2153.000000
1,0.991474,0.989064,0.990268,2469.000000
accuracy,0.989615,0.989615,0.989615,0.989615
macro avg,0.989484,0.989655,0.989568,4622.000000
weighted avg,0.989620,0.989615,0.989616,4622.000000
